# Xavier's Sandbox Notebook

**Conents:**
1. Conda environment testing.
2. Key raster types
2. Existing FCPG Tools Functions - Descriptions and Pseudo-Code.
3. Testing refactored code (later).

# Conda Environemnt Testing

## Branch: refactor_planning - `FCPGtools_env.yml` (September 9th, 2022)

This .yml installed successfully, but then was never able to resolve the environment to conda-forge install TauDem.
**Input:**
```
conda activate fcpgtools
conda install -c conda-forge taudem
```

**Output - very slow FAIL:**
```
Collecting package metadata (current_repodata.json): done
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Collecting package metadata (repodata.json): done
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: -
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
```

**ATTEMPT 2 with taudem in the .yml, as well as removing version specifications**

**Output - very slow FAIL:**
```
Solving environment: -
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
Examining conflict for jupyterlab geopandas pip conda-build python-language-server ipympl conda:
conflict for jupyterlab geopandas pip conda-build python-language-server ipympl conda: for python-language-server jupyterlab: : 24it [00:35,  1.66s/it] Examining conflict for geopandas conda-build python-language-server jupyter-lsp-python ipympl descartes: : 30it [00:40, Examining conflict for geopandas conda-build python-language-server jupyter-lsp-python ipympl descartes: : 31it [00:40, Examining conflict for geopandas taudem rasterio ipympl descartes: : 31it [00:42,  1.06s/it]                            Examining conflict for geopandas pip conda-build rasterio python-language-server ipympl descartes conda: : 34it [00:46, Examining conflict for geopandas pip conda-build rasterio python-language-server ipympl descartes conda: : 35it [00:46, Examining conflict for geopandas ipympl descartes python-language-server: : 35it [00:48,  1.14s/it]                     Examining conflict for pip conda-build python-language-server jupyter-lsp-python ipympl descartes conda: : 38it [00:52, Examining conflict for pip conda-build python-language-server jupyter-lsp-python ipympl descartes conda: : 39it [00:52, Examining conflict for pip conda-build jupyter-lsp-python ipympl descartes conda: : 39it [00:54,  1.29s/it]    
```

# Key Rasters

### ESRI NHDPlus Flow Direction Raster (FDR)
An example (from [`NHDPlusV21_MA_02_02a_FdrFac_01.7z`](https://www.epa.gov/waterdata/nhdplus-mid-atlantic-data-vector-processing-unit-02))of 8 bit flow direction raster (8 cardinal directions) using [NHDPlus](https://www.epa.gov/waterdata/get-nhdplus-national-hydrography-dataset-plus-data#v2datamap) raster coding convention (see below). 

**Cardinality:**
* 1 -> E (note that this is the only cardinal direction that does not need to be reclassified)
* 2 -> SE
* 4 -> S
* 8 -> SW
* 16 -> W
* 32 -> NW
* 64 -> N
* 128 -> NE

![Central PA FDR](imgs\esri_nhdplus_fdr_example.png)

# Existing FCPGTools Functions

In [3]:
#import FCPGtools as fc
import os
import rasterio as rs
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import subprocess

## Reclassify ESRI flow direction raster (FDR) values into TauDEM FDR compatible values.
**Existing function:** `tauDrainDir(inRast: str, outRast: str, brand=1, updateDict={...})`

```
Parameters
    inRast : str
        Path to a raster encoded with ESRI flow direction values.
    outRast : str
        Path to output a raster with flow directions encoded for TauDEM. File will be
        overwritten if it already exists.
    band : int (optional)
        Band to read the flow direction grid from if inRast is multiband, defaults to 1.
    updateDict : dict (optional)
        Dictionary of Rasterio raster options used to create outRast. Defaults have been supplied, 
        but may not work in all situations and input file formats.
    verbose : bool (optional)
        Print output, defaults to False.

    Returns
    -------
    outRast : raster
        Reclassified flow direction raster at the path specified above.
```

In [ ]:
def tauDrainDir(inRast, outRast, band=1, updateDict={}):
    # assert that inRast is a file, if not return 'inRast not found'

    # use rasterio to open inRast, assert that the specified band exists

    # save a copy of the input metadata via inRast.profile.copy()

    # make a tauDir .copy() of the input raster and remap HNDplus flow directions to Taudem flow direction!
    # i.e., 2 -> 8, 4 -> 7, etc. 

    # use updateDict to update the metadata

    # use rasterio to write tauDIR to the outRast .tif path

    return None

## Resample + reproject + clip a raster to match a FDR raster
**Existing function:** `resampleParam(inParam, fdr, outParam, resampleMethod="bilinear", cores=1, forceProj=False,
                  forceProj4="\"+proj=aea +lat_1=29.5 +lat_2=45.5 +lat_0=23 +lon_0=-96 +x_0=0 +y_0=0 +ellps=GRS80 +datum=NAD83 +units=m +no_defs\"",
                  verbose=False)`

```
Parameters
    ----------
    inParam : str
        Path to the input parameter data raster
    fdr : str
        Path to the flow direction raster
    outParam : str
        Path to the output file for the resampled parameter raster.
    resampleMethod : str (optional)
        resampling method, either 'bilinear' or 'near' for nearest neighbor. Bilinear should 
        generally be used for continuous data sets such as precipitation while nearest neighbor
        should generally be used for categorical 
        datasets such as land cover type. Defaults to bilinear.
    cores : int (optional)
        The number of cores to use. Defaults to 1.
    forceProj : bool (optional)
        Force the projection of the flow direction raster. This can be useful if the flow
        direction raster hasan unusual projection. Defaults to False. This parameter defaults
        to False; however, if set to True, forceProj4 must also be specified or the default
        proj4 string for USGS Albers will be used, see below.
    forceProj4 : str (optional)
        Proj4 string used to force the flow direction raster. This defaults to USGS Albers, but is not used 
        unless the forceProj parameter is set to True.
    verbose : bool (optional)
        Print output, defaults to False.

    Returns
    -------
    outParam : raster
        Resampled, reprojected, and clipped parameter raster.
```

**GDAL Warp Parameters:**
```
{'inParam': inParam,
                'outParam': outParam,
                'fdr': fdr,
                'cores': str(cores),
                'resampleMethod': resampleMethod,
                'xsize': xsize,
                'ysize': ysize,
                'fdrXmin': fdrXmin,
                'fdrXmax': fdrXmax,
                'fdrYmin': fdrYmin,
                'fdrYmax': fdrYmax,
                'fdrcrs': fdrcrs,
                'nodata': paramNoData,
                'datatype': outType}
```

**Note:** Currently cores defaults to 1, this requires users to know how many cores their computer has. It could be cool to check how many cores the computer has and use all of them if a boolean parameter is true (i.e., `optimize_cores:bool = True`).

In [ ]:
def resampleParam(inParam, fdr, outParam, resampleMethod="bilinear", cores=1, forceProj=False,
                  forceProj4="\"+proj=aea +lat_1=29.5 +lat_2=45.5 +lat_0=23 +lon_0=-96 +x_0=0 +y_0=0 +ellps=GRS80 +datum=NAD83 +units=m +no_defs\"",
                  verbose=False):
    # open in FDR raster with rasterop amd ge the coorindate system, cell size, and bounding coordinates

    # open the input raster and get the same attributes as the FDR raster (+ first band dtype)

    # get the output dtype. If it's 8-but, convert to 16but using a possibly outdated GDAL  `outType = 'Byte'`

    # check if resampling or repojection is required, if not, copy the metadata + raster as is

    # if things need to be changeg use a `gdalwarp` command line cmd via `subprocess.run(cmd, shell=True)`
    warpParams = {'dict of': 'gdal warp parameters'}

    cmd = 'gdalwarp -overwrite -tr {xsize} {ysize} -t_srs {fdrcrs} -te {fdrXmin} {fdrYmin} {fdrXmax} {fdrYmax} \
    -co "PROFILE=GeoTIFF" -co "TILED=YES" -co "SPARSE_OK=TRUE" -co "COMPRESS=LZW" -co "ZLEVEL=9" -co "NUM_THREADS={cores}" \
     -co "BIGTIFF=IF_SAFER" -r {resampleMethod} -dstnodata {nodata} -ot {datatype} {inParam} {outParam}'.format(
                **warpParams)

    result = subprocess.run(cmd, shell=True)
    result.stdout
    # note that the result.stdout output is calculated but not returned nor saved to a variable
    return None